#Importo librerie

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=45ec0ee27b2b13b4328e51a7b66ac901d2305a473ab4fa25364a504629ba4cf6
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=04edf6777da207623df6c430e6e84b414db48cd8381f340c557c2f444ba2f993
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [26]:
import pandas as pd 
import numpy as np 
import itertools

from time import time

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, r2_score, precision_recall_fscore_support, mean_squared_error
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.utils import shuffle

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

#import smogn

from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Dropout, Conv1D, Flatten, MaxPooling1D, AveragePooling1D, SimpleRNN, LSTM
from keras.regularizers import l2, l1, l1_l2
from keras.constraints import max_norm
from keras.callbacks import EarlyStopping, History
from keras import optimizers

from kerastuner import HyperModel
from kerastuner.tuners import Hyperband

#Definizione funzioni

In [52]:
# grafico matrice di confusione

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [53]:
# stampa tempo trascorso

def print_exec_time(start):
  print("Addestramento completato in %f secondi" % (time()-start))

In [54]:
# funzione per creare un sotto-df con le classi target bilanciate, in modo che anche le classi della variabile specificata nel parametro col siano egualmente rappresentate in ciascuna classe target

def undersample(df, label, col=None):
  hits = df[df[label] == 1]
  non_hits = df[df[label] == 0]

  if(col != None):

    non_hits_sampled_array = []

    for a in non_hits[col].unique():
      n = hits[hits[col] == a].id.count()
      non_hits_sampled_array.append(non_hits[non_hits[col] == a].sample(n))

    non_hits_sampled = pd.concat(non_hits_sampled_array)

  else:
    non_hits_sampled = df[df[label] == 0].sample(hits.shape[0])

  return pd.concat([hits,non_hits_sampled])

In [55]:
def over_under_balancing(X, Y, oversample_ratio, oversample_algorithm):
  '''
  oversample_algorithm --> [SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN]
  '''
  over = oversample_algorithm(sampling_strategy=oversample_ratio) # --> genero nuovi esempi nella classe in minoranza (hit) in modo da portarla al (es.) 50% della classe maggioritaria (non-hit)
  under = RandomUnderSampler(sampling_strategy=1) # --> rimuovo casualmente esempi dalla classe maggioritaria (non-hit) fino a portarla al 100% della classe minoritaria (hit)
  steps = [('o', over),('u', under)]
  pipeline = Pipeline(steps=steps)

  X, Y = pipeline.fit_resample(X, Y)
  
  return X, Y

In [56]:
# new_features_params
def insert_new_features_params(row, new_features_params):
    if(row.features == 'standard features'):
        return None
    else:
        return new_features_params

In [57]:
# selezionare il numero di componenti principali per LDA

def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

#Importo Dataset

In [58]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [59]:
drive.CreateFile({'id':'1-0o81KniM9hNtC8zqBaYyQWAmGdTYSS5'}).GetContentFile('dataset_final_4.0.csv')
df = pd.read_csv("dataset_final_4.0.csv").drop('Unnamed: 0',axis=1)

#DL

In [ ]:
category = 'deep learning'
problem = 'regression'
#oversample_algorithm = SMOTE
class_balancing = 'year-weighted undersample'

earlyStopping = EarlyStopping(monitor='loss', min_delta=0.001, patience=10, restore_best_weights=True)

result_df_array = []

x = 'all' # --> parametro da variare per considerare le new_features calcolate sui diversi intorni di anni --> [0,1,2,3,'all']

# # # FEATURES SELECT # # #

std_features_list = ['valence','acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','explicit','key_0','key_1','key_2','key_3','key_4','key_5','key_6','key_7','key_8','key_9','key_10','key_11','mode'] 

if(x == 'all'):
  new_features = ['valence_new','acousticness_new','danceability_new','duration_ms_new','energy_new','instrumentalness_new','liveness_new','loudness_new','speechiness_new','tempo_new','explicit_new','key_new','mode_new']
  new_features_list = []

  for i in range(4):
    for feat in new_features:
      new_features_list.append(feat + '_' + str(i))
else:
  new_features_list = ['valence_new_'+str(x),'acousticness_new_'+str(x),'danceability_new_'+str(x),'duration_ms_new_'+str(x),'energy_new_'+str(x),'instrumentalness_new_'+str(x),'liveness_new_'+str(x),'loudness_new_'+str(x),'speechiness_new_'+str(x),'tempo_new_'+str(x),'explicit_new_'+str(x),'key_new_'+str(x),'mode_new_'+str(x)]

extra_features = ['month','year','cos(month)','sin(month)','season_1','season_2','season_3','season_4', 'past_pop_n_hit', 'past_pop_n_weeks']
targets_list = ['hit', 'weeks_enc']

features_select = {'standard features': std_features_list+extra_features, 'standard + new features': std_features_list+new_features_list+extra_features, 'new features': new_features_list+extra_features}

for current_features in ['standard + new features', 'new features']: # features_select.keys()

    if(current_features != 'standard features'):
      new_features_params = x
    else:
      new_features_params = None

    # # # YEAR RANGE SELECT # # #

    year_range_select = [(1960,2020), (1960,1969), (1970,1979), (1980,1989), (1990,1999), (2000,2009), (2010,2020)]

    for year_range in year_range_select:

      # bilancio dataset
      df = undersample(df, 'hit', 'year_YYYY')

      year_start = year_range[0]
      year_end = year_range[1]

      # seleziono sotto_df
      mask_1 = df.year_YYYY >= year_start
      mask_2 = df.year_YYYY <= year_end
      sub_df = df[mask_1]
      sub_df = sub_df[mask_2]

      # regolo dimensione test set in base a numero di anni considerato
      if((year_end - year_start) > 10):
          test_size = 0.2
          val_size = int(0.05 * sub_df.shape[0])
      else:
          test_size = 0.3
          val_size = int(0.08 * sub_df.shape[0])
      
      # creo validation set
      sub_df = shuffle(sub_df)
      val_set = sub_df.iloc[:val_size].copy()
      sub_df = sub_df.iloc[val_size:].copy()

      # seleziono features correnti
      features = features_select[current_features]

      # creo array numpy
      X_val = val_set[features].drop(['month'], axis=1).values
      Y_val = val_set['weeks_enc'].values
      
      X = sub_df[features].drop(['month'], axis=1).values
      Y = sub_df['weeks_enc'].values
      
      # creo set addestramento e test
      X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)


      # ------------------------------------------- #

      # MODELLI

      # --- model_0 --- #

      model_0 = Sequential()
      # hidden layer
      model_0.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
      model_0.add(Dropout(0.75))
      # output layer
      model_0.add(Dense(1,activation='linear'))

      # --- model_1 --- #

      model_1 = Sequential()
      # hidden layer
      model_1.add(Dense(1024, input_dim=X_train.shape[1], activation='relu'))
      model_1.add(Dropout(0.75))
      # hidden layer
      model_1.add(Dense(254, activation='relu'))
      model_1.add(Dropout(0.65))
      # hidden layer
      model_1.add(Dense(128, activation='relu'))
      model_1.add(Dropout(0.65))
      # hidden layer
      model_1.add(Dense(64, activation='relu'))
      model_1.add(Dropout(0.6))
      # output layer
      model_1.add(Dense(1,activation='linear'))

      models = [model_0, model_1]

      for i, model in enumerate(models):

        adam = optimizers.Adam()  # learning_rate=learning_rates[i]

        model.compile(optimizer=adam, loss='mean_squared_error')
        print('\n')
        model.summary()

        time_0 = time()
        model.fit(X_train,Y_train, epochs=100, validation_data=(X_val, Y_val), batch_size=254, callbacks=[earlyStopping])
        print_exec_time(time_0)

        # METRICHE

        # test set
        Y_pred = model.predict(X_test) # effettuo predizioni

        # calcolo metriche
        MSE = mean_squared_error(Y_test, Y_pred)
        r2 = r2_score(Y_test, Y_pred)

        # train set
        Y_pred_train = model.predict(X_train) # effettuo predizioni

        # calcolo metriche
        MSE_train = mean_squared_error(Y_train, Y_pred_train)
        r2_train = r2_score(Y_train, Y_pred_train)

        # imposto a None le metriche della classificazione
        accuracy = None
        loss = None
        precision = [None, None]
        recall = [None, None]
        fscore = [None, None]
        conf_matrix = None
        accuracy_train = None
        loss_train = None
        precision_train = [None, None]
        recall_train = [None, None]
        fscore_train = [None, None]
        conf_matrix_train = None

        if(i == 1):
          parameters = '1 hidden layer (64)'
        else:
          parameters = '4 hidden layers (1024, 254, 128, 64)'

        tot_time = time() - time_0

        result_df = pd.DataFrame(
                [['MLP',
                parameters,
                class_balancing,
                problem,
                accuracy,
                loss,
                conf_matrix,
                precision[0],
                precision[1],
                recall[0],
                recall[1],
                fscore[0],
                fscore[1],
                accuracy_train,
                loss_train,
                conf_matrix_train,
                precision_train[0],
                precision_train[1],
                recall_train[0],
                recall_train[1],
                fscore_train[0],
                fscore_train[1],
                MSE,
                r2,
                MSE_train,
                r2_train,
                tot_time]],
          columns=['algorithm',
                  'parameters',
                  'class_balancing',
                  'problem',
                  'test_accuracy',
                  'test_log_loss',
                  'test_confusion_matrix',
                  'test_precision_0',
                  'test_precision_1', 
                  'test_recall_0',
                  'test_recall_1',
                  'test_fscore_0',
                  'test_fscore_1',
                  'train_accuracy',
                  'train_log_loss',
                  'train_confusion_matrix',
                  'train_precision_0',
                  'train_precision_1',
                  'train_recall_0',
                  'train_recall_1',
                  'train_fscore_0',
                  'train_fscore_1',
                  'test_MSE',
                  'test_r2',
                  'train_MSE',
                  'train_r2',
                  'tot_time'])
        

        # inserisco campi mancanti

        # year_range
        year_range_array = [year_range for i in range(result_df.shape[0])]
        result_df.insert(0, 'year_range', year_range_array)

        # features
        features_array = [current_features for i in range(result_df.shape[0])]
        result_df.insert(0, 'features', features_array)

        # new_features_params
        new_features_params_array = [new_features_params for i in range(result_df.shape[0])]
        result_df.insert(0, 'new_features_params', new_features_params_array)

        # category
        category_array = ['deep learning' for i in range(result_df.shape[0])]
        result_df.insert(0, 'category', category_array)
        
        result_df_array.append(result_df)

df_tot = pd.concat(result_df_array)



Model: "sequential_136"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_476 (Dense)            (None, 64)                5504      
_________________________________________________________________
dropout_340 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_477 (Dense)            (None, 1)                 65        
Total params: 5,569
Trainable params: 5,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
141/141 [==============================] - 1s 3ms/step - loss: 27.6747 - val_loss: 20.3290
Epoch 2/100
141/141 [==============================] - 0s 2ms/step - loss: 21.8498 - val_loss: 19.6735
Epoch 3/100
141/141 [==============================] - 0s 2ms/step - loss: 21.5114 - val_loss: 19.5008
Epoch 4/100
141/141 [==============================] - 0s 3ms/step - 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Boolean Series key will be reindexed to match DataFrame index.




Model: "sequential_140"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_490 (Dense)            (None, 64)                5504      
_________________________________________________________________
dropout_350 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_491 (Dense)            (None, 1)                 65        
Total params: 5,569
Trainable params: 5,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
15/15 [==============================] - 1s 16ms/step - loss: 37.4499 - val_loss: 25.2900
Epoch 2/100
15/15 [==============================] - 0s 3ms/step - loss: 27.3076 - val_loss: 21.4964
Epoch 3/100
15/15 [==============================] - 0s 3ms/step - loss: 25.1093 - val_loss: 21.1687
Epoch 4/100
15/15 [==============================] - 0s 3ms/step - loss: 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Boolean Series key will be reindexed to match DataFrame index.




Model: "sequential_142"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_497 (Dense)            (None, 64)                5504      
_________________________________________________________________
dropout_355 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_498 (Dense)            (None, 1)                 65        
Total params: 5,569
Trainable params: 5,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
15/15 [==============================] - 1s 14ms/step - loss: 44.4439 - val_loss: 32.4895
Epoch 2/100
15/15 [==============================] - 0s 3ms/step - loss: 31.4397 - val_loss: 24.6017
Epoch 3/100
15/15 [==============================] - 0s 3ms/step - loss: 26.7910 - val_loss: 22.4075
Epoch 4/100
15/15 [==============================] - 0s 3ms/step - loss: 2

...



Model: "sequential_158"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_553 (Dense)            (None, 64)                3968      
_________________________________________________________________
dropout_395 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_554 (Dense)            (None, 1)                 65        
Total params: 4,033
Trainable params: 4,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
19/19 [==============================] - 1s 11ms/step - loss: 38.1733 - val_loss: 27.5594
Epoch 2/100
19/19 [==============================] - 0s 3ms/step - loss: 27.8896 - val_loss: 22.9141
Epoch 3/100
19/19 [==============================] - 0s 3ms/step - loss: 24.7822 - val_loss: 22.0556
Epoch 4/100
19/19 [==============================] - 0s 3ms/step - loss: 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Boolean Series key will be reindexed to match DataFrame index.




Model: "sequential_160"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_560 (Dense)            (None, 64)                3968      
_________________________________________________________________
dropout_400 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_561 (Dense)            (None, 1)                 65        
Total params: 4,033
Trainable params: 4,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
26/26 [==============================] - 1s 9ms/step - loss: 35.8169 - val_loss: 24.6548
Epoch 2/100
26/26 [==============================] - 0s 3ms/step - loss: 24.9912 - val_loss: 21.8097
Epoch 3/100
26/26 [==============================] - 0s 3ms/step - loss: 24.1158 - val_loss: 21.5493
Epoch 4/100
26/26 [==============================] - 0s 3ms/step - loss: 23

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Boolean Series key will be reindexed to match DataFrame index.




Model: "sequential_162"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_567 (Dense)            (None, 64)                3968      
_________________________________________________________________
dropout_405 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_568 (Dense)            (None, 1)                 65        
Total params: 4,033
Trainable params: 4,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
34/34 [==============================] - 1s 7ms/step - loss: 30.7963 - val_loss: 20.6373
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 21.9074 - val_loss: 19.5344
Epoch 3/100
34/34 [==============================] - 0s 3ms/step - loss: 21.9658 - val_loss: 19.3274
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 21

In [ ]:
df_tot.sort_values('test_r2', ascending=False).head(50)

,category,new_features_params,features,year_range,algorithm,parameters,class_balancing,problem,test_accuracy,test_log_loss,test_confusion_matrix,test_precision_0,test_precision_1,test_recall_0,test_recall_1,test_fscore_0,test_fscore_1,train_accuracy,train_log_loss,train_confusion_matrix,train_precision_0,train_precision_1,train_recall_0,train_recall_1,train_fscore_0,train_fscore_1,test_MSE,test_r2,train_MSE,train_r2,tot_time
0,deep learning,all,new features,"(1980, 1989)",MLP,1 hidden layer (64),year-weighted undersample,regression,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,18.111715,0.172229,17.876539,0.186282,11.464724
0,deep learning,all,standard + new features,"(1960, 2020)",MLP,"4 hidden layers (1024, 254, 128, 64)",year-weighted undersample,regression,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,17.413362,0.169124,17.277101,0.171965,32.050600
0,deep learning,all,standard + new features,"(2010, 2020)",MLP,1 hidden layer (64),year-weighted undersample,regression,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,16.481524,0.164606,15.639564,0.203521,12.035303
0,deep learning,all,standard + new features,"(1960, 2020)",MLP,1 hidden layer (64),year-weighted undersample,regression,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,17.564111,0.161932,17.326923,0.169577,43.612438
0,deep learning,all,new features,"(2000, 2009)",MLP,1 hidden layer (64),year-weighted undersample,regression,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,17.855511,0.161853,18.087367,0.160148,11.979798
0,deep learning,all,standard + new features,"(1980, 1989)",MLP,1 hidden layer (64),year-weighted undersample,regression,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,18.321210,0.160942,16.962868,0.228245,8.001913
0,deep learning,all,new features,"(1960, 2020)",MLP,1 hidden layer (64),year-weighted undersample,regression,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,17.601727,0.157239,17.707027,0.152280,47.586623
0,deep learning,all,standard + new features,"(2000, 2009)",MLP,1 hidden layer (64),year-weighted undersample,regression,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,18.394729,0.147528,16.997400,0.205242,21.803133
0,deep learning,all,new features,"(1960, 2020)",MLP,"4 hidden layers (1024, 254, 128, 64)",year-weighted undersample,regression,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,17.927970,0.141619,18.028353,0.136897,32.771489
0,deep learning,all,standard + new features,"(1990, 1999)",MLP,1 hidden layer (64),year-weighted undersample,regression,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,18.738167,0.134945,17.128402,0.215418,11.502775


# Export

In [ ]:
from google.colab import  drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

df_tot.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/results_DL_4.1_'+str(x)+'_regression.csv')

Mounted at /content/drive
